In [1]:
import os
import nibabel as nib
import numpy as np
import json
import pandas as pd

In [4]:
excel_path = 'D:\\OneDrive - Imperial College London\\Documents\\Student Projects\\Kaihe Zhang\\PairedData.xlsx'

df = pd.read_excel(excel_path)
sample_ids = df.iloc[:, 0].astype(str).tolist()

root_folder = 'D:\\OneDrive - Imperial College London\\Documents\\Student Projects\\Kaihe Zhang\\TestingData'


In [5]:
seg_file_dict = {}
for dirpath, dirnames, filenames in os.walk(root_folder):
    for filename in filenames:
        if filename.startswith('seg') and filename.endswith('.nii.gz'):
            seg_file_dict[filename] = os.path.join(dirpath, filename)

In [ ]:
def get_bbox_from_mask(mask_path):
    mask_img = nib.load(mask_path)
    mask_data = mask_img.get_fdata()
    coords = np.argwhere(mask_data > 0)
    if coords.size == 0:
        raise ValueError(f"No segmented region found in {mask_path}")
    x_min, y_min, z_min = coords.min(axis=0)
    x_max, y_max, z_max = coords.max(axis=0)
    return {
        "x_min": int(x_min), "x_max": int(x_max + 1),
        "y_min": int(y_min), "y_max": int(y_max + 1),
        "z_min": int(z_min), "z_max": int(z_max + 1)
    }

bbox_coords = {}
for sample_id in sample_ids:
    seg_filename = f"seg{sample_id}.nii.gz"
    seg_path = seg_file_dict.get(seg_filename, None)
    if seg_path is not None:
        try:
            bbox = get_bbox_from_mask(seg_path)
            bbox_coords[seg_filename] = bbox
        except Exception as e:
            print(f"Error for {sample_id}: {e}")
    else:
        print(f"[WARN] No seg file found for {sample_id}")

with open('1bbox_coords.json', 'w') as f:
    json.dump(bbox_coords, f, indent=2, ensure_ascii=False)

print('Bounding Box 坐标已保存至 bbox_coords.json 文件中。')
